In [1]:
import numpy as np
import pandas as pd

import graphviz

In [2]:
columns = ["no", "id", "name", "status", "mother_1", "mother_2", "daughter_1", "daughter_2", "colour_1", "colour_2", "p_x", "p_y", "p_z", "e", "m"]

start_line = 225  
end_line = 861   
num_rows = end_line - start_line

df_pythia = pd.read_csv("mymain488_c8_p_14N_proj1PeV.log", delim_whitespace=True, header=None, names=columns, index_col=False, skiprows=start_line, nrows=num_rows)

display(df_pythia)

,no,id,name,status,mother_1,mother_2,daughter_1,daughter_2,colour_1,colour_2,p_x,p_y,p_z,e,m
0,0,90,(system),-11,0,0,0,0,0,0,0.000,0.000,-32963.912,38072.258,19049.339
1,1,2212,(p+),-12,0,0,3,0,0,0,0.000,0.000,2554.171,2554.172,0.938
2,2,1000070140,(14N),-12,0,0,4,0,0,0,0.000,0.000,-35518.084,35518.086,13.047
3,3,2212,(p+),-13,1,0,153,0,0,0,0.000,0.000,2554.171,2554.172,0.938
4,4,2212,(p+),-13,2,0,154,0,0,0,0.000,0.000,-2554.171,2554.172,0.938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,631,221,eta,84,579,591,0,0,0,0,0.267,0.871,-3.285,3.452,0.548
632,632,221,eta,84,579,591,0,0,0,0,0.353,0.486,-2.652,2.773,0.548
633,633,221,eta,84,579,591,0,0,0,0,-0.422,0.472,-8.460,8.501,0.548
634,634,323,K*+,84,579,591,0,0,0,0,0.397,1.422,-6.231,6.455,0.808


In [3]:
df_final_state = df_pythia[df_pythia["status"] > 0]

display(df_final_state)

,no,id,name,status,mother_1,mother_2,daughter_1,daughter_2,colour_1,colour_2,p_x,p_y,p_z,e,m
168,168,22,gamma,62,152,152,0,0,0,0,0.075,-0.257,0.435,0.510,0.000
386,386,1000050119,NucRem,14,2,0,0,0,0,0,0.000,0.000,-27855.569,27855.571,10.232
402,402,111,pi0,81,400,401,0,0,0,0,0.122,0.047,11.009,11.011,0.135
409,409,-211,pi-,83,407,408,0,0,0,0,-0.020,-0.328,-129.424,129.424,0.140
410,410,221,eta,83,407,408,0,0,0,0,0.425,0.761,-1451.855,1451.855,0.548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,631,221,eta,84,579,591,0,0,0,0,0.267,0.871,-3.285,3.452,0.548
632,632,221,eta,84,579,591,0,0,0,0,0.353,0.486,-2.652,2.773,0.548
633,633,221,eta,84,579,591,0,0,0,0,-0.422,0.472,-8.460,8.501,0.548
634,634,323,K*+,84,579,591,0,0,0,0,0.397,1.422,-6.231,6.455,0.808


In [4]:
system_row = df_pythia[df_pythia["name"].str.contains("\(system\)", regex=True)]
initial_system_ecm = system_row["e"].values[0]
print("Initial energy of Pythia event's (system): ", initial_system_ecm)

rows_3_4 = df_pythia[df_pythia["no"].isin([3, 4])]
initial_ecm = rows_3_4["e"].sum()
print("Initial energy of Pythia event's: ", initial_ecm)

total_ecm = df_final_state["e"].sum()
print("Total center-of-mass energy of the final state particles: ", total_ecm)

nucrem_row = df_pythia[df_pythia["name"].str.contains("NucRem", regex=False)]
nucrem_e_value = nucrem_row["e"].values[0]
total_ecm -= nucrem_e_value
print("Total center-of-mass energy of the final state particles without the nuclear remnant: ", total_ecm)

Initial energy of Pythia event's (system):  38072.258
Initial energy of Pythia event's:  5108.344
Total center-of-mass energy of the final state particles:  38072.253
Total center-of-mass energy of the final state particles without the nuclear remnant:  10216.681999999997


### Genealogy

In [5]:
def trace_genealogy(particle_no, df, genealogy=None, depth=5):
    if genealogy is None:
        genealogy = []
    
    particle = df[df['no'] == particle_no]
    
    if particle.empty or depth == 0:
        return genealogy
    
    genealogy.append({
        "no": int(particle['no']),
        "name": str(particle['name'].values[0])
    })
    
    mother_1 = int(particle['mother_1'])
    mother_2 = int(particle['mother_2'])
    
    if mother_1 > 0:
        trace_genealogy(mother_1, df, genealogy, depth - 1)
    if mother_2 > 0:
        trace_genealogy(mother_2, df, genealogy, depth - 1)
    
    return genealogy

final_particles = df_pythia[df_pythia['status'] > 0]['no'].values
genealogies = {fp: trace_genealogy(fp, df_pythia) for fp in final_particles}

def generate_flowchart(genealogies, df):
    dot = graphviz.Digraph(comment="Particle Genealogy", engine='dot')

    dot.attr(size='10,10', rankdir='LR', splines='true', nodesep='0.2', ranksep='0.3')

    # list of final state particles
    final_particles = df[df['status'] > 0]['no'].values

    for final_no, genealogy in genealogies.items():
        # iterate over all particles in the genealogy
        for i in range(len(genealogy)):
            particle = genealogy[i]

            # highlight initial particles (no = 3 and 4) in red
            if particle['no'] in [3, 4]:
                dot.node(str(particle['no']), particle['name'], style='filled', fillcolor='red', width='0.3', height='0.3')
            # highlight final state particles (status > 0) in green
            elif particle['no'] in final_particles:
                dot.node(str(particle['no']), particle['name'], style='filled', fillcolor='green', width='0.3', height='0.3')
            else:
                dot.node(str(particle['no']), particle['name'], width='0.3', height='0.3')

            if i > 0:
                mother = genealogy[i - 1]
                dot.edge(str(mother['no']), str(particle['no']))

    return dot

dot = generate_flowchart(genealogies, df_pythia)
dot.render('particle_genealogy_flowchart_highlight', format='svg') 

/tmp/ipykernel_11340/2529970196.py:11: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  "no": int(particle['no']),
/tmp/ipykernel_11340/2529970196.py:15: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  mother_1 = int(particle['mother_1'])
/tmp/ipykernel_11340/2529970196.py:16: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  mother_2 = int(particle['mother_2'])


'particle_genealogy_flowchart_highlight.svg'

### Boost using Lorentz transformation manually (based on the boost velocity calculated from the initial particle momenta)

In [6]:
def lorentz_boost(df_final_state, df_pythia):
    # initial state particles from df_pythia
    particle_1 = df_pythia[df_pythia["no"] == 3].iloc[0]
    particle_2 = df_pythia[df_pythia["no"] == 4].iloc[0]
    
    # momenta and energies of the initial state particles
    p1_x, p1_y, p1_z = particle_1["p_x"], particle_1["p_y"], particle_1["p_z"]
    p2_x, p2_y, p2_z = particle_2["p_x"], particle_2["p_y"], particle_2["p_z"]
    
    E1, E2 = particle_1["e"], particle_2["e"]
    
    # total momentum and energy in the lab frame
    P_x = p1_x + p2_x
    P_y = p1_y + p2_y
    P_z = p1_z + p2_z
    E_total = E1 + E2
    
    # boost velocity vector (v/c)
    beta_x = P_x / E_total if E_total != 0 else 0
    beta_y = P_y / E_total if E_total != 0 else 0
    beta_z = P_z / E_total if E_total != 0 else 0
    
    #  magnitude of boost velocity and gamma factor
    beta_squared = beta_x**2 + beta_y**2 + beta_z**2
    if beta_squared >= 1:
        raise ValueError("Beta squared is greater than or equal to 1, which is non-physical.")
    gamma = 1.0 / np.sqrt(1.0 - beta_squared)
    
    # apply Lorentz boost to a four-momentum vector
    def apply_lorentz_boost(p_x, p_y, p_z, E):
        beta_dot_p = beta_x * p_x + beta_y * p_y + beta_z * p_z
        
        # if beta is zero
        if beta_squared == 0:
            return p_x, p_y, p_z, E
        
        # boosted energy and momentum components
        E_boosted = gamma * (E - beta_dot_p)
        p_x_boosted = p_x + (gamma - 1) * beta_dot_p * beta_x / beta_squared - gamma * beta_x * E
        p_y_boosted = p_y + (gamma - 1) * beta_dot_p * beta_y / beta_squared - gamma * beta_y * E
        p_z_boosted = p_z + (gamma - 1) * beta_dot_p * beta_z / beta_squared - gamma * beta_z * E
        
        return p_x_boosted, p_y_boosted, p_z_boosted, E_boosted
    
    # apply boost to each final state particle
    df_final_state_boosted = df_final_state.copy()
    for index, row in df_final_state.iterrows():
        p_x, p_y, p_z, E = row["p_x"], row["p_y"], row["p_z"], row["e"]
        p_x_boosted, p_y_boosted, p_z_boosted, E_boosted = apply_lorentz_boost(p_x, p_y, p_z, E)
        
        df_final_state_boosted.at[index, "p_x"] = p_x_boosted
        df_final_state_boosted.at[index, "p_y"] = p_y_boosted
        df_final_state_boosted.at[index, "p_z"] = p_z_boosted
        df_final_state_boosted.at[index, "e"] = E_boosted
    
    return df_final_state_boosted


In [7]:
df_final_state_boosted = lorentz_boost(df_final_state, df_pythia)
display(df_final_state_boosted)

,no,id,name,status,mother_1,mother_2,daughter_1,daughter_2,colour_1,colour_2,p_x,p_y,p_z,e,m
168,168,22,gamma,62,152,152,0,0,0,0,0.075,-0.257,0.435,0.510,0.000
386,386,1000050119,NucRem,14,2,0,0,0,0,0,0.000,0.000,-27855.569,27855.571,10.232
402,402,111,pi0,81,400,401,0,0,0,0,0.122,0.047,11.009,11.011,0.135
409,409,-211,pi-,83,407,408,0,0,0,0,-0.020,-0.328,-129.424,129.424,0.140
410,410,221,eta,83,407,408,0,0,0,0,0.425,0.761,-1451.855,1451.855,0.548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,631,221,eta,84,579,591,0,0,0,0,0.267,0.871,-3.285,3.452,0.548
632,632,221,eta,84,579,591,0,0,0,0,0.353,0.486,-2.652,2.773,0.548
633,633,221,eta,84,579,591,0,0,0,0,-0.422,0.472,-8.460,8.501,0.548
634,634,323,K*+,84,579,591,0,0,0,0,0.397,1.422,-6.231,6.455,0.808


In [8]:
total_boosted_lab = df_final_state_boosted["e"].sum()
print("Total boosted to the lab frame energy of the final state particles: ", total_boosted_lab)

Total boosted to the lab frame energy of the final state particles:  38072.253


### Boost using `inverseBoost_` matrix from Corsika 8

Extract of COMBoost.inl in Corsika 8:
```
inline void COMBoost::setBoost(double const coshEta, double const sinhEta) {
    boost_ << coshEta, sinhEta, sinhEta, coshEta;
    inverseBoost_ << coshEta, -sinhEta, -sinhEta, coshEta;
}
```

In [9]:
def set_boost_matrix(eta):
    coshEta = np.cosh(eta)
    sinhEta = np.sinh(eta)
    inverseBoost_ = np.array([
        [coshEta, -sinhEta, 0, 0],
        [-sinhEta, coshEta, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    return inverseBoost_

In [10]:
def lorentz_boost_with_matrix(df_final_state, inverseBoost_):
    # apply inverse Lorentz boost using inverseBoost_ matrix as in Corsika 8
    def apply_lorentz_boost_matrix(p_x, p_y, p_z, E):
        p4 = np.array([E, p_z, p_y, p_x])
        
        # apply inverse boost using matrix multiplication
        p4_boosted = np.dot(inverseBoost_, p4)
        
        E_boosted = p4_boosted[0]
        p_z_boosted = p4_boosted[1]
        p_y_boosted = p4_boosted[2]
        p_x_boosted = p4_boosted[3]
        
        return p_x_boosted, p_y_boosted, p_z_boosted, E_boosted
    
    # apply boost to each final state particle
    df_final_state_boosted = df_final_state.copy()
    for index, row in df_final_state.iterrows():
        p_x, p_y, p_z, E = row["p_x"], row["p_y"], row["p_z"], row["e"]
        p_x_boosted, p_y_boosted, p_z_boosted, E_boosted = apply_lorentz_boost_matrix(p_x, p_y, p_z, E)
        
        df_final_state_boosted.at[index, "p_x"] = p_x_boosted
        df_final_state_boosted.at[index, "p_y"] = p_y_boosted
        df_final_state_boosted.at[index, "p_z"] = p_z_boosted
        df_final_state_boosted.at[index, "e"] = E_boosted
    
    return df_final_state_boosted

In [11]:
def get_beta_z(df_pythia):
    # initial state particles from df_pythia
    particle_1 = df_pythia[df_pythia["no"] == 3].iloc[0]
    particle_2 = df_pythia[df_pythia["no"] == 4].iloc[0]
    
    # momenta and energies of the initial state particles
    p1_z = particle_1["p_z"]
    p2_z = particle_2["p_z"]
    
    E1, E2 = particle_1["e"], particle_2["e"]
    
    # total momentum and energy in the lab frame
    P_z = p1_z + p2_z
    E_total = E1 + E2
    
    # boost velocity in the z direction (v_z/c)
    beta_z = p2_z / E2 if E_total != 0 else 0
    
    return beta_z

In [12]:
eta = np.arctanh(get_beta_z(df_pythia))
print("From the boost velocity, η = ", eta)

inverseBoost_ = set_boost_matrix(eta)

df_final_state_boosted = lorentz_boost_with_matrix(df_final_state, inverseBoost_)
display(df_final_state_boosted)

total_boosted_lab = df_final_state_boosted["e"].sum()
print(f"Total boosted to the lab frame energy of the final state particles: {total_boosted_lab:3.2e}")
total_boosted_lab = df_final_state_boosted["e"].sum() - df_final_state_boosted[df_final_state_boosted["name"].str.contains("NucRem")]["e"].sum()
print(f"Total boosted to the lab frame energy of the final state particles, without the nuclear remnant: {total_boosted_lab:3.2e}")

From the boost velocity, η =  -7.723192821559363


,no,id,name,status,mother_1,mother_2,daughter_1,daughter_2,colour_1,colour_2,p_x,p_y,p_z,e,m
168,168,22,gamma,62,152,152,0,0,0,0,0.075,-0.257,1067.927650,1067.927683,0.000
386,386,1000050119,NucRem,14,2,0,0,0,0,0,0.000,0.000,-10.064413,14.584741,10.232
402,402,111,pi0,81,400,401,0,0,0,0,0.122,0.047,24884.409759,24884.409760,0.135
409,409,-211,pi-,83,407,408,0,0,0,0,-0.020,-0.328,-0.057263,0.057263,0.140
410,410,221,eta,83,407,408,0,0,0,0,0.425,0.761,-0.642367,0.642367,0.548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,631,221,eta,84,579,591,0,0,0,0,0.267,0.871,188.722235,188.725216,0.548
632,632,221,eta,84,579,591,0,0,0,0,0.353,0.486,136.738745,136.741145,0.548
633,633,221,eta,84,579,591,0,0,0,0,-0.422,0.472,46.329618,46.337122,0.548
634,634,323,K*+,84,579,591,0,0,0,0,0.397,1.422,253.135603,253.141216,0.808


Total boosted to the lab frame energy of the final state particles: 5.77e+06
Total boosted to the lab frame energy of the final state particles, without the nuclear remnant: 5.77e+06


In [13]:
df_final_state_boosted

,no,id,name,status,mother_1,mother_2,daughter_1,daughter_2,colour_1,colour_2,p_x,p_y,p_z,e,m
168,168,22,gamma,62,152,152,0,0,0,0,0.075,-0.257,1067.927650,1067.927683,0.000
386,386,1000050119,NucRem,14,2,0,0,0,0,0,0.000,0.000,-10.064413,14.584741,10.232
402,402,111,pi0,81,400,401,0,0,0,0,0.122,0.047,24884.409759,24884.409760,0.135
409,409,-211,pi-,83,407,408,0,0,0,0,-0.020,-0.328,-0.057263,0.057263,0.140
410,410,221,eta,83,407,408,0,0,0,0,0.425,0.761,-0.642367,0.642367,0.548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,631,221,eta,84,579,591,0,0,0,0,0.267,0.871,188.722235,188.725216,0.548
632,632,221,eta,84,579,591,0,0,0,0,0.353,0.486,136.738745,136.741145,0.548
633,633,221,eta,84,579,591,0,0,0,0,-0.422,0.472,46.329618,46.337122,0.548
634,634,323,K*+,84,579,591,0,0,0,0,0.397,1.422,253.135603,253.141216,0.808


In [14]:
gamma = np.sqrt(1 + (1e15 / 938.27e6) ** 2)
eta = np.arccosh(gamma)
print("From the Lorentz factor of a 1 PeV proton, η = ", eta)

inverseBoost_ = set_boost_matrix(eta)

df_final_state_boosted = lorentz_boost_with_matrix(df_final_state, inverseBoost_)
display(df_final_state_boosted)

total_boosted_lab = df_final_state_boosted["e"].sum()
print(f"Total boosted to the lab frame energy of the final state particles: {total_boosted_lab:3.2e}")
total_boosted_lab = df_final_state_boosted["e"].sum() - df_final_state_boosted[df_final_state_boosted["name"].str.contains("NucRem")]["e"].sum()
print(f"Total boosted to the lab frame energy of the final state particles, without the nuclear remnant: {total_boosted_lab:3.2e}")

From the Lorentz factor of a 1 PeV proton, η =  14.572375263438326


,no,id,name,status,mother_1,mother_2,daughter_1,daughter_2,colour_1,colour_2,p_x,p_y,p_z,e,m
168,168,22,gamma,62,152,152,0,0,0,0,0.075,-0.257,-7.993435e+04,7.993435e+04,0.000
386,386,1000050119,NucRem,14,2,0,0,0,0,0,0.000,0.000,-5.937645e+10,5.937645e+10,10.232
402,402,111,pi0,81,400,401,0,0,0,0,0.122,0.047,-2.131583e+03,2.131583e+03,0.135
409,409,-211,pi-,83,407,408,0,0,0,0,-0.020,-0.328,-2.758779e+08,2.758779e+08,0.140
410,410,221,eta,83,407,408,0,0,0,0,0.425,0.761,-3.094749e+09,3.094749e+09,0.548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,631,221,eta,84,579,591,0,0,0,0,0.267,0.871,-7.180236e+06,7.180236e+06,0.548
632,632,221,eta,84,579,591,0,0,0,0,0.353,0.486,-5.781918e+06,5.781918e+06,0.548
633,633,221,eta,84,579,591,0,0,0,0,-0.422,0.472,-1.807689e+07,1.807689e+07,0.548
634,634,323,K*+,84,579,591,0,0,0,0,0.397,1.422,-1.352063e+07,1.352063e+07,0.808


Total boosted to the lab frame energy of the final state particles: 7.57e+10
Total boosted to the lab frame energy of the final state particles, without the nuclear remnant: 1.63e+10


In [15]:
e_proton = np.sqrt((1e15 * 3e8) ** 2 + 938.27e6 ** 2)
s = 938.27e6**2 + 14 * 931.494e6**2 + 2 * e_proton * 14 * 931.494e6
gamma_lab = np.sqrt(s) / (2 * 14 * 931.494e6)
eta = np.arccosh(gamma_lab)
print("sqrt(s) = ", np.sqrt(s))
print("From the Lorentz factor of a 1 PeV proton onto a fixed-target of nitrogen-14, γ = ", gamma_lab,", η = ", eta)

inverseBoost_ = set_boost_matrix(eta)

df_final_state_boosted = lorentz_boost_with_matrix(df_final_state, inverseBoost_)
display(df_final_state_boosted)

total_boosted_lab = df_final_state_boosted["e"].sum()
print(f"Total boosted to the lab frame energy of the final state particles: {total_boosted_lab:3.2e}")
total_boosted_lab = df_final_state_boosted["e"].sum() - df_final_state_boosted[df_final_state_boosted["name"].str.contains("NucRem")]["e"].sum()
print(f"Total boosted to the lab frame energy of the final state particles, without the nuclear remnant: {total_boosted_lab:3.2e}")

sqrt(s) =  8.84564842168171e+16
From the Lorentz factor of a 1 PeV proton onto a fixed-target of nitrogen-14, γ =  3391498.1208688524 , η =  15.729929485857232


,no,id,name,status,mother_1,mother_2,daughter_1,daughter_2,colour_1,colour_2,p_x,p_y,p_z,e,m
168,168,22,gamma,62,152,152,0,0,0,0,0.075,-0.257,-2.543624e+05,2.543624e+05,0.000
386,386,1000050119,NucRem,14,2,0,0,0,0,0,0.000,0.000,-1.889442e+11,1.889442e+11,10.232
402,402,111,pi0,81,400,401,0,0,0,0,0.122,0.047,-6.782996e+03,6.782996e+03,0.135
409,409,-211,pi-,83,407,408,0,0,0,0,-0.020,-0.328,-8.778825e+08,8.778825e+08,0.140
410,410,221,eta,83,407,408,0,0,0,0,0.425,0.761,-9.847927e+09,9.847927e+09,0.548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,631,221,eta,84,579,591,0,0,0,0,0.267,0.871,-2.284852e+07,2.284852e+07,0.548
632,632,221,eta,84,579,591,0,0,0,0,0.353,0.486,-1.839888e+07,1.839888e+07,0.548
633,633,221,eta,84,579,591,0,0,0,0,-0.422,0.472,-5.752320e+07,5.752320e+07,0.548
634,634,323,K*+,84,579,591,0,0,0,0,0.397,1.422,-4.302455e+07,4.302455e+07,0.808


Total boosted to the lab frame energy of the final state particles: 2.41e+11
Total boosted to the lab frame energy of the final state particles, without the nuclear remnant: 5.20e+10


In [16]:

eta = np.arccosh(15)
print("From tip-toeing using γ = 15, η = arctanh(γ) = ", eta)

inverseBoost_ = set_boost_matrix(eta)

df_final_state_boosted = lorentz_boost_with_matrix(df_final_state, inverseBoost_)
display(df_final_state_boosted)

total_boosted_lab = df_final_state_boosted["e"].sum()
print(f"Total boosted to the lab frame energy of the final state particles: {total_boosted_lab:3.2e}")
total_boosted_lab = df_final_state_boosted["e"].sum() - df_final_state_boosted[df_final_state_boosted["name"].str.contains("NucRem")]["e"].sum()
print(f"Total boosted to the lab frame energy of the final state particles, without the nuclear remnant: {total_boosted_lab:3.2e}")

From tip-toeing using γ = 15, η = arctanh(γ) =  3.4000844141133397


,no,id,name,status,mother_1,mother_2,daughter_1,daughter_2,colour_1,colour_2,p_x,p_y,p_z,e,m
168,168,22,gamma,62,152,152,0,0,0,0,0.075,-0.257,-1.107981,1.139516,0.000
386,386,1000050119,NucRem,14,2,0,0,0,0,0,0.000,0.000,-834737.546980,834737.547047,10.232
402,402,111,pi0,81,400,401,0,0,0,0,0.122,0.047,0.337442,0.397375,0.135
409,409,-211,pi-,83,407,408,0,0,0,0,-0.020,-0.328,-3878.401063,3878.401063,0.140
410,410,221,eta,83,407,408,0,0,0,0,0.425,0.761,-43507.200941,43507.200941,0.548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,631,221,eta,84,579,591,0,0,0,0,0.267,0.871,-100.939805,100.945378,0.548
632,632,221,eta,84,579,591,0,0,0,0,0.353,0.486,-81.282464,81.286502,0.548
633,633,221,eta,84,579,591,0,0,0,0,-0.422,0.472,-254.131318,254.132686,0.548
634,634,323,K*+,84,579,591,0,0,0,0,0.397,1.422,-190.074594,190.082069,0.808


Total boosted to the lab frame energy of the final state particles: 1.06e+06
Total boosted to the lab frame energy of the final state particles, without the nuclear remnant: 2.30e+05
